In [ ]:
!pip3 install wikipedia-api

In [ ]:
import wikipediaapi
import pandas as pd 
from wikiextractor.clean import clean_markup

def get_categorymembers(categorymembers, level=0, max_level=1, root_pages=None):
    if root_pages is None:
        root_pages = []

    for c in categorymembers.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            get_categorymembers(c.categorymembers, level=level + 1, max_level=max_level, root_pages=root_pages)
        else:
            if c.title.startswith('Category'):
                try:
                    title  = c.title.split(":")[1]
                except:
                    continue
            else:
                title = c.title
            root_pages.append(title)
    return root_pages

def get_root_pages_from_category(category_name):
    cat = wiki_wiki.page(f"Category:{category_name}")
    return get_categorymembers(cat.categorymembers)

# Example usage:

API_KEY = 'ProjName (email)'
wiki_wiki = wikipediaapi.Wikipedia(API_KEY, 'en')

domains = ["Statistics", "Artificial intelligence", "Computational mathematics", "Numerical analysis", "Applied mathematics", "Probability"]
root_pages = []
for domain in domains:
    root_page = get_root_pages_from_category(domain)
    root_pages += root_page

root_pages = set(root_pages)

In [ ]:
wiki_data = {"topic":[], "text":[]}
for page in root_pages:
   wiki_data["topic"].append(page)
   wiki_data["text"].append(wiki_wiki.page(page).text)

wiki_df = pd.DataFrame(wiki_data)
wiki_df = wiki_df.query("text.str.len() != 0")
wiki_df = wiki_df.reset_index(drop=True)
wiki_df.to_parquet('wiki_ml.parquet')

In [1]:


def find_noise_section(strings):
    for index, string in enumerate(strings):
        if string.startswith("See also.") or string.startswith("External links.") or string.startswith("References.") or string.startswith("Bibliography."):
            return index
    return -1

def clean_text(text):
    
    paragraphs = list(clean_markup(text, ignore_headers=False))
    
    # text = [paragraph for paragraph in paragraphs if len(paragraph) > 3]
    text = []
    for paragraph in paragraphs:
        if len(paragraph) > 5 or len(text) == 0:
            text.append(paragraph)
        else:
            text[-1] = text[-1] + " " + " ".join(paragraph.split(" "))
    text = "\n".join(text)
    
    index = find_noise_section(text.split("## "))
    if index!= -1:
        text = text.split("## ")[:index]
        return " ##".join(text)
    
    return text



wiki_df = pd.read_parquet('./data/wiki_ml_mediawiki.parquet')
wiki_df.text = wiki_df.text.apply(clean_text)
wiki_df["id"] = wiki_df.index
wiki_df.to_parquet('./data/wiki_ml_mediawiki_cleaned.parquet')